In [136]:
import openai
import logging
import torch
import time
from datetime import datetime
# from espnet2.bin.tts_inference import Text2Speech
from IPython.display import display, Audio

In [137]:
# Model Path
# train_config_file = 'D:/Project3180/kan-bayashi_ljspeech_vits/exp/tts_train_vits_raw_phn_tacotron_g2p_en_no_space/config.yaml'
# model_file = 'D:/Project3180/kan-bayashi_ljspeech_vits/exp/tts_train_vits_raw_phn_tacotron_g2p_en_no_space/train.total_count.ave_10best.pth'

In [138]:
# load_tts_model
# model = Text2Speech(train_config_file,model_file)

Two scores are defined here: one is the emotion score, and the other is the extreme behavior score. For the former, negative emotions decrease the score, while positive emotions increase it, with each adjustment typically ranging between 0 and 1. For the latter, the score increases by 1 each time an extreme behavior is detected.

In [139]:
# Set up OpenAI API key
api_key = open("API_Key.txt","r").read()
openai.api_key = api_key
#定义两个分数
emotion_score = 0
extreme_behavior_score = 0

the skin suit code

In [140]:
def tts_synthesis(x):
    global model
    # synthesis
    start_time = time.time()
    print('Synthesis...')
    with torch.no_grad():
#         start = time.time()
        wav = model(x)["wav"]
#     rtf = (time.time() - start) / (len(wav) / text2speech.fs)
#     print(f"RTF = {rtf:5f}")
    # let us listen to generated samples
    print('Inference Completed. Time Used: {}'.format(time.time() - start_time))
    audio = Audio(wav.view(-1).cpu().numpy(), rate=model.fs *1.07, autoplay = True)
    print('Audio Generated. Time Used: {}'.format(time.time() - start_time))
    display(audio)
    # save audio
    with open('./audios/{}.wav'.format('_'.join(x.split(' ')[:5])), 'wb') as f:
        print('Saving Audio...')
        f.write(audio.data)
        print("Audio Saved.")

In [141]:
def send_message(message_log,p_max_tokens = 300):
    # Use OpenAI's ChatCompletion API to get the chatbot's response
    # print('Sending msg to ChatGPT and waiting for the response...')
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # The name of the OpenAI chatbot model to use
        messages=message_log,   # The conversation history up to this point, as a list of dictionaries
        max_tokens=p_max_tokens,        # The maximum number of tokens (words or subwords) in the generated response
        stop = None,              # The stopping sequence for the generated response, if any (not used here)
#         temperature=0.7,        # The "creativity" of the generated response (higher temperature = more creative)
    )
    # print('Response Received. Time Used: {}'.format(time.time() - start_time))
    return response['choices'][0]['message']['content']

In [142]:
def dump_dialogue(p_message_log):
    print("Saving Dialogue...")
    # dd/mm/YY H:M
    dt_string = datetime.now().strftime("%d_%m_%Y-%H_%M")
    with open('./dialogues/{}.txt'.format(dt_string), 'w') as filehandle:
        for x in p_message_log:
            filehandle.write(str(x) + "\n")
    print("Dialogue Saved.")

using gpt-2

In [143]:
import torch
from transformers import GPT2Tokenizer, GPT2Model
torch.manual_seed(0)

model_name = "gpt2" 
model = GPT2Model.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model.eval()

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

This is the code to derive the positive and negative sentiment scores of an input. The official documentation states that the positive (or negative) sentiment score of an input should range from 0 to 1, but the actual result is between 0 and -1. There's an issue where intense emotions cause both scores to rise simultaneously, influencing each other. Therefore, the next step is to evaluate the sentiment individually. If the sentiment is judged as positive, add the positive sentiment score minus -1 (since it's between -1 and 0, the closer to 0, the more intense the emotion). For negative sentiment, subtract the negative sentiment score minus -1.  

Note: The return value is a dictionary.

In [144]:
def predict_sentiment(text):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    input_ids = torch.tensor(tokens).unsqueeze(0)

    with torch.no_grad():
        outputs = model(input_ids)
        last_hidden_states = outputs[0]

    sentiment_scores = last_hidden_states.mean(dim=1).squeeze()
    positive_score = sentiment_scores[1].item()  
    negative_score = sentiment_scores[0].item()
    return {"positive": positive_score, "negative": negative_score}


The code for judging emotions. The nltk library is imported. The negative_list is used to detect negative words. A double negative means no negation, while a single negative means negation. If there are negative words, it returns -1; if there are positive words, it returns 0; if neither is present, it returns 0. For its usage, refer to the above context

In [145]:
import nltk
from nltk.corpus import opinion_lexicon

nltk.download('opinion_lexicon')
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())
negation_list = ["not", "no", "never", "none", "nobody", "nothing", "neither", "nowhere", "hardly", "barely", "rarely"]


def has_motion_word(sentence):
    words = nltk.word_tokenize(sentence.lower())
    word_pos = nltk.pos_tag(words)
    negation_count = 0
    for i in range(len(word_pos)):
        # detect negation
        if word_pos[i][0] in negation_list:
                negation_count += 1
    for word in words:
        if word in negative_words:
            return (-1)**(negation_count%2 + 1)
        elif word in positive_words:
            return (-1)**(negation_count%2 + 2)
    return 0


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\119282\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


Code for detecting suicide and self-harm. Customize the suicide_list. The function returns 1 if extreme behavior is detected, otherwise 0. Negations are also considered.

In [146]:
suicide_list = ["suicide","murder","self-destruction","self-murder","martyrdom","self-slaughter","assassination","homicide","hari-kari","killing","slaying","manslaughter",
"euthanasia","rubout","chance-medley","massacre","kill","slaughter","slay","scrag","croak","annihilate","assassinate","decimate","smite","martyr","euthanize","euthanatize"]

def has_suicide_word(sentence):
    words = nltk.word_tokenize(sentence.lower())
    word_pos = nltk.pos_tag(words)

    negation_count = 0
    for i in range(len(word_pos)):
        # detect negation
        if word_pos[i][0] in negation_list:
            negation_count += 1
    for word in words:
        if word in suicide_list:
            return (negation_count%2 + 1)%2
    return 0

The simple therapeutic function. The logic is to pause sending to GPT-3, then pop up a simple MBSR therapy. Print line by line, and print the next line after the user presses enter. After it ends, ask the user if they feel better. If they feel better, divide the two scores (emotion and extremity) by two; otherwise, leave them unchanged.

In [147]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
def cure_function():
    global emotion_score
    global extreme_behavior_score
    mbsr_1 = ["Lie with your back on the floor or a bed and close your eyes.",
    "Move your awareness through your body, focusing on one area at a time.", 
    "Stop whenever you find an area that is unusually tight or sore and focus your breath on this area until it relaxes.",
    "You can use a calm and healing visualization at this point as well (e.g., a ball of white light melting into the sore spot)"]
    print("Well, maybe you need the following MBSR exercise:")
    for i in mbsr_1:
        print(i)
        input("Press enter to continue:")
    print("Do you feel better now?")
    user_input = input()

    if sid.polarity_scores(user_input)['compound'] > 0:
        emotion_score /= 2
        extreme_behavior_score /= 2
    print("You:", user_input)
    print("Emotion_score:", emotion_score, "  extreme_behavior_score: ", extreme_behavior_score)
    print("Alright. Do you have any other thorghts you'd like to share?")
    return

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\119282\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


main function

In [148]:

def main():
    global emotion_score#衡量情绪的标准。正加负减
    global extreme_behavior_score#极端行为分，判断到一个加一分
    cure_flag = False#判断有没有治疗过，默认为false
    # Initialize the conversation history with a message from the chatbot
    message_log = [
        {'role': 'system', 'content': 'You are a helpful and caring AI therapist, who gives suggestion and mental support with concise answers.'},
    ]

    # Start a loop that runs until the user types "quit"
    while True:
            print("Emotion_score:", emotion_score, "  extreme_behavior_score:", extreme_behavior_score)#在每次输入前打出情绪分（因为这个东西要展示出来的，不是真的给用户用的）
            if emotion_score < -2 or extreme_behavior_score >= 1:#达到治疗的值。
                if cure_flag == False:#如果没有治疗过
                    cure_function()
                    cure_flag = True#说明治疗过了
            if emotion_score < -4 or extreme_behavior_score >= 3:#每个句子正面和负面值在0到1之间。暂时设定为阙值为4。
                if emotion_score <-4:#这个if和else是将emotion_score从0到-4映射到0到100.具体的，如果在0到-4之间就对应着0到80. 如果大于-4（设定的极限是-5.5），则对应的是80到100.
                    emotion_score = emotion_score*80/(-4)
                else:
                    emotion_score = (-emotion_score-4)*20/1.5 + 80
                extreme_behavior_score = (extreme_behavior_score/3)*100#极端分的映射。0到3对应到0到100
                print("\nYou are not well. Your depression score is:%f and your extreme behavior score is:%f. "%(emotion_score, extreme_behavior_score))#打印两个分
                print("Maybe you need to take a depression test: https://www.mind-diagnostics.org/depression-test")#心里测评网站。在输出这个链接的时候整个程序已经结束了。
                break
            print()

            # If this is not the first request, get the user's input and add it to the conversation history
            user_input = input("You: ")

            # If the user types "quit", end the loop and print a goodbye message
            if user_input.lower() == "quit":#如果退出也要打印两个分。换算代码是一样的
                if emotion_score <-4:
                    emotion_score = emotion_score*80/(-4)
                else:
                    emotion_score = (-emotion_score-4)*20/1.5 + 80
                extreme_behavior_score = (extreme_behavior_score/3)*100
                print("Your depression score is:%f and your extreme behavior score is:%f."%(emotion_score, extreme_behavior_score))
                print("Goodbye!")
                # dump_dialogue(message_log)
                break

            message_log.append({"role": "user", "content": user_input})
            
            try:
                # Send the conversation history to the chatbot and get its response
                response = send_message(message_log)

                #emotion_score algorithm
                return_dict = predict_sentiment(user_input)#返回值是一个字典
                return_list = [return_dict["negative"], -1, return_dict["positive"]]#将字典转化为list

                tmp_emotion = has_motion_word(user_input)#如果有情绪词就计分。这里如果没有情绪词返回0，下面就是0.如果负面情绪这里就是-1，下面就是减。如果正面情绪这里就是1，下面就是加。
                emotion_score += tmp_emotion*(return_list[tmp_emotion+1]+1)#参考上一句话的输出。return_list[tmp_emotion+1]是对应的情绪分。+1是-（-1），参考上面def predict_sentiment的注释。
                
                #extreme_behavior_score algorithm
                if has_suicide_word(user_input) == 1:#检测到极端行为词就加一
                    extreme_behavior_score += 1  
            except:#处理错误。一般不会发生
                logging.exception('..')
                print("FATAL: Exiting.")
                # dump_dialogue(message_log)
                break
            # Add the chatbot's response to the conversation history and print it to the console
            message_log.append({"role": "assistant", "content": response})
            print(f"AI assistant: {response}")
            # tts_synthesis(response)

In [149]:
# tts_synthesis("I am an AI therapist and I am happy to help you. You are always welcome to share whatever you feel comfortable with. Sometimes it can be helpful to talk to someone who is neutral and non-judgmental about what's on your mind. I am here to offer support, encouragement, and guidance as you navigate through this difficult time.")

In [150]:
main()

Emotion_score: 0   extreme_behavior_score: 0

You:  i am not feeling well these days
AI assistant: I'm sorry to hear that. Would you like to share what's been troubling you? Sometimes it helps to talk about what's going on.
Emotion_score: -1.3203484117984772   extreme_behavior_score: 0

Your depression score is:44.271312 and your extreme behavior score is:0.000000.
Goodbye!
